Runs the FIR model fits for the models defined in workflow #6 [6_define-FIR-models](http://localhost:8888/notebooks/8_define-FIR-models.ipynb).

Note: **you need to have run the function redef_FIRs.m, located on scratch, on the outputs of workflow #8, prior to running this notebook**

In [1]:
from bids.layout import BIDSLayout 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html
#
# set spm path to copy with use edit of spm_fMRI_design.m [see: https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=spm;f525092b.1508]
# scratch/qbi/uqkgarn1
matlab_cmd = '/scratch/qbi/uqkgarn1/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

210309-13:40:15,912 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()

fitfirs = pe.Workflow(name="fitfirs") # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


# grab and sink data

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                            outfields=['spm']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'spm': [['sub','TR']]}
dg.inputs.field_template = {'spm': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-%s/TR%s/FLGLM/SPM.mat'}

In [4]:
# debugging
dg.inputs.sub = '01'
dg.inputs.TR = '1510'
res = dg.run()
res.outputs

210309-13:40:29,218 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpz8dfk_us/data-grabber".
210309-13:40:29,223 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210309-13:40:29,229 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



spm = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR1510/FLGLM/SPM.mat

In [5]:
inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                   name='info')
inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
# inf.iterables = [('sub', ['01']), ('TR', ['1510'])]
inf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0]
    TR = [t for t in l if re.search('TR([0-9]*)',t)][-1]
    name = [name, TR, "FLGLM"]
    name = '/'.join(name)
    return name

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_gzip_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# estimate model

In [8]:
# EstimateModel - estimate the parameters of the model
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

# zip files

from .nii -> .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs

In [10]:
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

# Connect workflow

In [11]:
fitfirs.connect([(inf, dg, [('sub',  'sub')]),
                 (inf, dg, [('TR', 'TR')]),
                 (dg, ds, [(('spm', printSubPath), 
                            'container')]),
                 (dg, estimate, [('spm', 'spm_mat_file')]),
                 (estimate, gzip, [('beta_images', 'in_file')]),
                 (gzip, ds, [('out_file', 'PARAMS')])
])

In [ ]:
fitfirs.run()

210309-13:40:29,405 nipype.workflow INFO:
	 Workflow fitfirs settings: ['check', 'execution', 'logging', 'monitoring']
210309-13:40:29,477 nipype.workflow INFO:
	 Running serially.
210309-13:40:29,478 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.data-grabber" in "/tmp/tmpz8dfk_us/data-grabber".
210309-13:40:29,484 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210309-13:40:29,489 nipype.workflow INFO:
	 [Node] Finished "fitfirs.data-grabber".
210309-13:40:29,490 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.estimate" in "/tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate".
210309-13:40:29,520 nipype.workflow INFO:
	 [Node] Running "estimate" ("nipype.interfaces.spm.model.EstimateModel")
210309-13:48:07,208 nipype.workflow INFO:
	 [Node] Finished "fitfirs.estimate".
210309-13:48:07,210 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.gzip" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip".
210309-13:48:07,233 nipype.workflow INFO:

210309-13:48:11,87 nipype.workflow INFO:
	 [Node] Finished "_gzip17".
210309-13:48:11,89 nipype.workflow INFO:
	 [Node] Setting-up "_gzip18" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip18".
210309-13:48:11,93 nipype.workflow INFO:
	 [Node] Running "_gzip18" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0019.nii
210309-13:48:11,291 nipype.workflow INFO:
	 [Node] Finished "_gzip18".
210309-13:48:11,293 nipype.workflow INFO:
	 [Node] Setting-up "_gzip19" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip19".
210309-13:48:11,296 nipype.workflow INFO:
	 [Node] Running "_gzip19" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0020.nii
210309-13:48:11,488 nipype.workflow INFO:
	 [Node] Finished "_gzip19".
210309-13:48:11,490 nipype.workflow INFO:
	 [Node] Setting-up "_gzip20" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_su

210309-13:48:15,305 nipype.workflow INFO:
	 [Node] Finished "_gzip37".
210309-13:48:15,307 nipype.workflow INFO:
	 [Node] Setting-up "_gzip38" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip38".
210309-13:48:15,310 nipype.workflow INFO:
	 [Node] Running "_gzip38" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0039.nii
210309-13:48:15,509 nipype.workflow INFO:
	 [Node] Finished "_gzip38".
210309-13:48:15,511 nipype.workflow INFO:
	 [Node] Setting-up "_gzip39" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip39".
210309-13:48:15,515 nipype.workflow INFO:
	 [Node] Running "_gzip39" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0040.nii
210309-13:48:15,710 nipype.workflow INFO:
	 [Node] Finished "_gzip39".
210309-13:48:15,712 nipype.workflow INFO:
	 [Node] Setting-up "_gzip40" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920

210309-13:48:19,504 nipype.workflow INFO:
	 [Node] Finished "_gzip57".
210309-13:48:19,506 nipype.workflow INFO:
	 [Node] Setting-up "_gzip58" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip58".
210309-13:48:19,510 nipype.workflow INFO:
	 [Node] Running "_gzip58" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0059.nii
210309-13:48:19,707 nipype.workflow INFO:
	 [Node] Finished "_gzip58".
210309-13:48:19,709 nipype.workflow INFO:
	 [Node] Setting-up "_gzip59" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip59".
210309-13:48:19,713 nipype.workflow INFO:
	 [Node] Running "_gzip59" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0060.nii
210309-13:48:19,918 nipype.workflow INFO:
	 [Node] Finished "_gzip59".
210309-13:48:19,921 nipype.workflow INFO:
	 [Node] Setting-up "_gzip60" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920

210309-13:48:23,682 nipype.workflow INFO:
	 [Node] Finished "_gzip77".
210309-13:48:23,685 nipype.workflow INFO:
	 [Node] Setting-up "_gzip78" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip78".
210309-13:48:23,689 nipype.workflow INFO:
	 [Node] Running "_gzip78" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0079.nii
210309-13:48:23,905 nipype.workflow INFO:
	 [Node] Finished "_gzip78".
210309-13:48:23,907 nipype.workflow INFO:
	 [Node] Setting-up "_gzip79" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip79".
210309-13:48:23,911 nipype.workflow INFO:
	 [Node] Running "_gzip79" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmp6xj6q55v/fitfirs/_TR_1920_sub_05/estimate/beta_0080.nii
210309-13:48:24,113 nipype.workflow INFO:
	 [Node] Finished "_gzip79".
210309-13:48:24,115 nipype.workflow INFO:
	 [Node] Setting-up "_gzip80" in "/tmp/tmpn2jg9is2/fitfirs/_TR_1920

...

210309-15:19:09,984 nipype.workflow INFO:
	 [Node] Finished "_gzip66".
210309-15:19:09,986 nipype.workflow INFO:
	 [Node] Setting-up "_gzip67" in "/tmp/tmpfzhq7t8q/fitfirs/_TR_1510_sub_03/gzip/mapflow/_gzip67".
210309-15:19:09,991 nipype.workflow INFO:
	 [Node] Running "_gzip67" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpzdv0wfl7/fitfirs/_TR_1510_sub_03/estimate/beta_0068.nii
210309-15:19:10,268 nipype.workflow INFO:
	 [Node] Finished "_gzip67".
210309-15:19:10,270 nipype.workflow INFO:
	 [Node] Setting-up "_gzip68" in "/tmp/tmpfzhq7t8q/fitfirs/_TR_1510_sub_03/gzip/mapflow/_gzip68".
210309-15:19:10,275 nipype.workflow INFO:
	 [Node] Running "_gzip68" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpzdv0wfl7/fitfirs/_TR_1510_sub_03/estimate/beta_0069.nii
210309-15:19:10,553 nipype.workflow INFO:
	 [Node] Finished "_gzip68".
210309-15:19:10,555 nipype.workflow INFO:
	 [Node] Setting-up "_gzip69" in "/tmp/tmpfzhq7t8q/fitfirs/_TR_1510

210309-15:19:16,30 nipype.workflow INFO:
	 [Node] Finished "_gzip86".
210309-15:19:16,32 nipype.workflow INFO:
	 [Node] Setting-up "_gzip87" in "/tmp/tmpfzhq7t8q/fitfirs/_TR_1510_sub_03/gzip/mapflow/_gzip87".
210309-15:19:16,36 nipype.workflow INFO:
	 [Node] Running "_gzip87" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpzdv0wfl7/fitfirs/_TR_1510_sub_03/estimate/beta_0088.nii
210309-15:19:16,291 nipype.workflow INFO:
	 [Node] Finished "_gzip87".
210309-15:19:16,293 nipype.workflow INFO:
	 [Node] Setting-up "_gzip88" in "/tmp/tmpfzhq7t8q/fitfirs/_TR_1510_sub_03/gzip/mapflow/_gzip88".
210309-15:19:16,298 nipype.workflow INFO:
	 [Node] Running "_gzip88" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpzdv0wfl7/fitfirs/_TR_1510_sub_03/estimate/beta_0089.nii
210309-15:19:16,576 nipype.workflow INFO:
	 [Node] Finished "_gzip88".
210309-15:19:16,578 nipype.workflow INFO:
	 [Node] Setting-up "_gzip89" in "/tmp/tmpfzhq7t8q/fitfirs/_TR_1510_su

210309-15:19:18,56 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0022.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0022.nii.gz
210309-15:19:18,67 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0023.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0023.nii.gz
210309-15:19:18,78 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0024.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0024.nii.gz
210309-15:19:18,88 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0025.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAM

210309-15:19:18,393 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0055.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0055.nii.gz
210309-15:19:18,404 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0056.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0056.nii.gz
210309-15:19:18,414 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0057.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0057.nii.gz
210309-15:19:18,424 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0058.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/P

210309-15:19:18,720 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0088.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0088.nii.gz
210309-15:19:18,729 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0089.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0089.nii.gz
210309-15:19:18,770 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0090.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS//beta_0090.nii.gz
210309-15:19:18,828 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/PARAMS/_TR_1510_sub_03/beta_0091.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-03/TR1510/FLGLM/P